In [88]:
from scipy.linalg import qr, cholesky, pinv, solve, norm
from numpy.random import randn, rand
from numpy.linalg import lstsq, eigvalsh
import numpy as np

In [35]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
# X[:,3] = X[:,2]

# Understanding the pseudoinverse

In [36]:
# form pseudoinverse
Xd = pinv(X)

In [37]:
# X†X ≈ I_d
Xd @ X

array([[ 1.00000000e+00,  1.21325658e-15, -5.93354903e-16,
        -5.22569701e-16],
       [ 1.63390620e-16,  1.00000000e+00,  3.49192965e-16,
        -2.96482188e-17],
       [-2.24863718e-16, -1.77875162e-16,  1.00000000e+00,
         2.08881750e-16],
       [ 2.16871217e-16,  7.86582834e-16, -4.72861238e-16,
         1.00000000e+00]])

In [38]:
np.allclose(Xd @ X, np.identity(4))

True

In [39]:
# XX† !≈ I_n
X @ Xd

array([[ 0.9248032 ,  0.11448589, -0.12772691, -0.01725997, -0.18661504,
        -0.0706964 ],
       [ 0.11448589,  0.8082757 ,  0.29677779, -0.00583499,  0.20967187,
         0.0989226 ],
       [-0.12772691,  0.29677779,  0.18214885,  0.15928191,  0.12024443,
        -0.0689201 ],
       [-0.01725997, -0.00583499,  0.15928191,  0.9368442 , -0.1800615 ,
        -0.03228498],
       [-0.18661504,  0.20967187,  0.12024443, -0.1800615 ,  0.21874955,
        -0.21267312],
       [-0.0706964 ,  0.0989226 , -0.0689201 , -0.03228498, -0.21267312,
         0.9291785 ]])

In [40]:
np.allclose(X @ Xd, np.identity(6))

False

In [41]:
Q,R = qr(X)
Q,R = qr(X, mode='economic')

In [42]:
np.allclose(X, Q @ R)

True

In [43]:
Q

array([[-4.86365563e-02, -6.47395955e-01, -5.76663051e-01,
        -4.13250393e-01],
       [ 2.87531592e-01,  9.47934881e-03,  3.29462942e-01,
        -7.85471575e-01],
       [ 2.95162678e-01,  1.76161410e-01,  1.46580956e-01,
        -2.06177164e-01],
       [ 4.23842280e-01,  6.04249889e-01, -6.19658653e-01,
        -9.00397114e-02],
       [ 2.45277032e-01, -7.03608799e-02,  3.91727744e-01,
        -1.36913590e-02],
       [-7.66835029e-01,  4.23895266e-01, -6.68437928e-04,
        -4.01814499e-01]])

In [44]:
R

array([[ 1.11312779,  0.24356613, -0.53663007, -1.00916615],
       [ 0.        ,  2.02062798, -1.51899367, -0.7970204 ],
       [ 0.        ,  0.        , -1.65963705, -0.32369515],
       [ 0.        ,  0.        ,  0.        ,  1.49871313]])

In [45]:
print(np.allclose(Q.T @ Q, np.identity(Q.shape[1])))
Q.T @ Q

True


array([[1.00000000e+00, 1.26792505e-16, 1.09105457e-16, 7.54070614e-17],
       [1.26792505e-16, 1.00000000e+00, 6.56988706e-17, 3.91868159e-16],
       [1.09105457e-16, 6.56988706e-17, 1.00000000e+00, 1.25473289e-16],
       [7.54070614e-17, 3.91868159e-16, 1.25473289e-16, 1.00000000e+00]])

In [46]:
# form data from noisy linear model
wtrue = randn(d)
y = X.dot(wtrue) + .01*randn(n)

In [47]:
# solve least squares problem to estimate w
Q,R = qr(X, mode='economic')
w = solve(R, Q.T @ y)

In [48]:
# how good is our estimate?
norm(w - wtrue)

0.013699048850174042

In [49]:
# compute mean square error
def mse(y,z):
    return sum((y-z)**2)/len(y)
    
mse(y,X.dot(w))

1.1082810525817631e-06

In [50]:
# we can use the numpy.lstsq call instead
w_lstsq = np.linalg.lstsq(X, y, rcond=None)[0]
norm(w_lstsq - w)

8.3820000221454525e-16

# Compute QR by hand

In [51]:
n,d = X.shape 
X0 = X.copy()
R = np.zeros((n,d))
Q = np.zeros((n,n))

# first column of Q points in direction of first column of X
r = norm(X[:,0])
Q[:,0] = X[:,0]/r
Q

array([[-0.04863656,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.28753159,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.29516268,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.42384228,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.24527703,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.76683503,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

In [52]:
# ensure Q*R matches X on first column
R[0,0] = r

In [53]:
# verify Q*R matches X in first column
(Q@R - X)[:,0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.11022302e-16])

In [54]:
# now delete that part from X; we've covered it already
X[:,0] -= Q[:,0]*R[0,0]

In [55]:
# verify Q*R + X = X0
np.isclose(Q@R + X, X0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [56]:
# eliminate component of other columns in direction of first column of Q 
for j in range(1,d):
    R[0,j] = Q[:,0].dot(X[:,j])
    X[:,j] -= Q[:,0]*R[0,j]
R

array([[ 1.11312779,  0.24356613, -0.53663007, -1.00916615],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [57]:
# verify Q*R + X = X0
np.isclose(Q@R + X, X0)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [58]:
# now for all the columns!
X = X0.copy()
Q *= 0
R *= 0

# compute the QR decomposition
for i in range(d):
    r = norm(X[:,i])
    Q[:,i] = X[:,i]/r
    for j in range(i,d):
        R[i,j] = Q[:,i].dot(X[:,j])
        X[:,j] -= Q[:,i]*R[i,j]
    print("iteration",i,": QR + X = X0?", np.isclose(Q@R + X, X0).all())

iteration 0 : QR + X = X0? True
iteration 1 : QR + X = X0? True
iteration 2 : QR + X = X0? True
iteration 3 : QR + X = X0? True


In [59]:
Q

array([[-4.86365563e-02, -6.47395955e-01,  5.76663051e-01,
        -4.13250393e-01,  0.00000000e+00,  0.00000000e+00],
       [ 2.87531592e-01,  9.47934881e-03, -3.29462942e-01,
        -7.85471575e-01,  0.00000000e+00,  0.00000000e+00],
       [ 2.95162678e-01,  1.76161410e-01, -1.46580956e-01,
        -2.06177164e-01,  0.00000000e+00,  0.00000000e+00],
       [ 4.23842280e-01,  6.04249889e-01,  6.19658653e-01,
        -9.00397114e-02,  0.00000000e+00,  0.00000000e+00],
       [ 2.45277032e-01, -7.03608799e-02, -3.91727744e-01,
        -1.36913590e-02,  0.00000000e+00,  0.00000000e+00],
       [-7.66835029e-01,  4.23895266e-01,  6.68437928e-04,
        -4.01814499e-01,  0.00000000e+00,  0.00000000e+00]])

In [60]:
R

array([[ 1.11312779,  0.24356613, -0.53663007, -1.00916615],
       [ 0.        ,  2.02062798, -1.51899367, -0.7970204 ],
       [ 0.        ,  0.        ,  1.65963705,  0.32369515],
       [ 0.        ,  0.        ,  0.        ,  1.49871313],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [61]:
"""Our very own QR function to compute the economy QR"""
def ourQR(X0):
    X = X0.copy()
    n,d = X.shape
    R = np.zeros((n,d))
    Q = np.zeros((n,n))

    # compute the QR decomposition
    for i in range(d):
        r = norm(X[:,i])
        Q[:,i] = X[:,i]/r
        for j in range(i,d):
            R[i,j] = Q[:,i].dot(X[:,j])
            X[:,j] -= Q[:,i]*R[i,j]
    return Q,R

In [62]:
# solve least squares problem to estimate w
Q,R = ourQR(X0)
w_byhand = solve(R[:d,:d], (Q.T @ y)[:d])

In [63]:
norm(w_byhand - w)

7.63656720759043e-16

# The importance of the permutation

In [140]:
X0 = np.zeros((3,3))
X0[:,0] = [1,1,0]
X0[:,1] = [0,1,0]
X0[:,2] = [0,1,1]
X0

Y = X0.copy()
Y[:,1] = X0[:,0]
Y[:,0] = X0[:,1]

In [144]:
# try X0 vs Y 
X = X0.copy()
# X = Y.copy()
X

array([[1., 0., 0.],
       [1., 1., 1.],
       [0., 0., 1.]])

In [145]:
# now for all the columns!
n,d = X.shape
Q = np.zeros((n,n))
R = np.zeros((n,d))

# compute the QR decomposition
for i in range(d):
    r = norm(X[:,i])
    Q[:,i] = X[:,i]/r
    for j in range(i,d):
        R[i,j] = Q[:,i].dot(X[:,j])
        X[:,j] -= Q[:,i]*R[i,j]
    print("iteration",i,": QR + X = X0?", np.isclose(Q@R + X, X0).all())

iteration 0 : QR + X = X0? True
iteration 1 : QR + X = X0? True
iteration 2 : QR + X = X0? True


In [146]:
Q,R

(array([[ 7.07106781e-01, -7.07106781e-01, -5.55111512e-17],
        [ 7.07106781e-01,  7.07106781e-01,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
 array([[1.41421356, 0.70710678, 0.70710678],
        [0.        , 0.70710678, 0.70710678],
        [0.        , 0.        , 1.        ]]))

The number of nonzeros in Q and R is larger using the first permutation (X0) than the second (Y). Why?

In [147]:
zero = 1e-16
(Q>zero).sum(), (R>zero).sum()

(4, 6)

In [148]:
X0, Y

(array([[1., 0., 0.],
        [1., 1., 1.],
        [0., 0., 1.]]),
 array([[0., 1., 0.],
        [1., 1., 1.],
        [0., 0., 1.]]))

# Same, but symmetric

In [76]:
np.random.seed(0)
X0 = np.zeros((5,5))
X0[:,0] = [rand(),0,0,0,0]
X0[:,1] = [0,rand(),0,0,0]
X0[:,2] = [rand(),0,rand(),0,0]
X0[:,3] = [rand(),rand(),0,0,0]
X0[:,4] = [0,0,rand(),0,rand()]
X0 = X0 + X0.T + np.eye(5)
eigvalsh(X0)

array([0.62140537, 1.48865546, 2.46040092, 2.72872784, 3.10212851])

In [79]:
def swap(X0,i,j):
    Y = X0.copy()
    Y[:,i] = X0[:,j]
    Y[:,j] = X0[:,i]
    return Y

def swap_sym(X0,i,j):
    Y = X0.copy()
    Y[:,i] = X0[:,j]
    Y[:,j] = X0[:,i]
    Y1 = Y.copy()
    Y1[i,:] = Y[j,:]
    Y1[j,:] = Y[i,:]
    return Y1

array([[2.09762701, 0.        , 0.        , 0.4236548 , 0.60276338],
       [0.        , 2.43037873, 0.        , 0.64589411, 0.        ],
       [0.        , 0.        , 2.783546  , 0.        , 0.43758721],
       [0.4236548 , 0.64589411, 0.        , 1.        , 0.        ],
       [0.60276338, 0.        , 0.43758721, 0.        , 2.08976637]])

In [115]:
Y = X0.copy()
# Y = swap_sym(X0, 2, 4) # try turning this on and off
Y

array([[2.09762701, 0.        , 0.        , 0.4236548 , 0.60276338],
       [0.        , 2.43037873, 0.        , 0.64589411, 0.        ],
       [0.        , 0.        , 2.783546  , 0.        , 0.43758721],
       [0.4236548 , 0.64589411, 0.        , 1.        , 0.        ],
       [0.60276338, 0.        , 0.43758721, 0.        , 2.08976637]])

In [116]:
R = cholesky(Y)
R

array([[ 1.44831868,  0.        ,  0.        ,  0.2925149 ,  0.41618145],
       [ 0.        ,  1.5589672 ,  0.        ,  0.41430898,  0.        ],
       [ 0.        ,  0.        ,  1.66839624,  0.        ,  0.26228015],
       [ 0.        ,  0.        ,  0.        ,  0.86184865, -0.14125366],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.35196741]])

In [117]:
# check it's a factorization 
R.T@R - Y

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         5.55111512e-17,  0.00000000e+00],
       [ 0.00000000e+00,  4.44089210e-16,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  4.44089210e-16,
         0.00000000e+00, -5.55111512e-17],
       [ 5.55111512e-17,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.21273863e-17],
       [ 0.00000000e+00,  0.00000000e+00, -5.55111512e-17,
         1.21273863e-17,  0.00000000e+00]])

In [118]:
# fill-in 
(R!=0) & (Y==0)

array([[False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False,  True],
       [False, False, False, False, False]])